In [104]:
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os
from fredapi import Fred
warnings.filterwarnings('ignore')

import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
import pmdarima as pm
from pmdarima import model_selection

%matplotlib inline

In [86]:
start = datetime(2000,1,31)
end = datetime.now()

In [87]:
reit_df_name_list = ['AMT', 'CUBE', 'ELS', 'PLD', 'FR']
reit_df_name_list2 = ['MAA', 'SUI', 'BXMT', 'RHP']

In [97]:
AMT = web.DataReader('AMT', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')

In [115]:
fred = Fred(api_key = 'aec5f13aa40d1af4edd587ffccceeb1a')
libor = round(fred.get_series('USD1MTD156N', start, end), 2)